# Import Necessary libraries

In [1]:
import pandas as pd

# load data

In [2]:
df=pd.read_csv("C:\\Users\\akhil\\Downloads\\heart.csv")

In [5]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
df.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

In [4]:
df.shape

(918, 12)

In [5]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [6]:
df.dtypes

Age                 int64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64
ExerciseAngina     object
Oldpeak           float64
ST_Slope           object
HeartDisease        int64
dtype: object

In [7]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [8]:
import numpy as np
df_num=df.select_dtypes(include=[np.number])
df_cat=df.select_dtypes(include=['object']) 

In [9]:
df_num.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
0,40,140,289,0,172,0.0,0
1,49,160,180,0,156,1.0,1
2,37,130,283,0,98,0.0,0
3,48,138,214,0,108,1.5,1
4,54,150,195,0,122,0.0,0


In [10]:
df_cat.head()

,Sex,ChestPainType,RestingECG,ExerciseAngina,ST_Slope
0,M,ATA,Normal,N,Up
1,F,NAP,Normal,N,Flat
2,M,ATA,ST,N,Up
3,F,ASY,Normal,Y,Flat
4,M,NAP,Normal,N,Up


In [11]:
df_cat.columns

Index(['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], dtype='object')

In [13]:
df_cat['HeartDisease']=df_num['HeartDisease']

In [14]:
df_cat.columns

Index(['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

# LabelEncoding

In [32]:
from sklearn.preprocessing import LabelEncoder

In [33]:
le=LabelEncoder()

In [34]:
df1=df.apply(le.fit_transform)

In [36]:
df1.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,12,1,1,41,147,0,1,98,0,10,2,0
1,21,0,2,55,40,0,1,82,0,20,1,1
2,9,1,1,31,141,0,2,25,0,10,2,0
3,20,0,0,39,72,0,1,34,1,25,1,1
4,26,1,2,49,53,0,1,48,0,10,2,0


# Chi-Square Test

In [37]:
from scipy.stats import chi2_contingency

In [38]:
chi2_results = {}
for col in ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']:
    contingency_table = pd.crosstab(df_cat[col], df_cat['HeartDisease'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    chi2_results[col] = p

In [39]:
chi2_df = pd.DataFrame(list(chi2_results.items()), columns=['Feature', 'P-Value'])
chi2_df['Significant'] = chi2_df['P-Value'] < 0.05  # Mark significant relationships
print(chi2_df)

          Feature       P-Value  Significant
0             Sex  4.597617e-20         True
1   ChestPainType  8.083728e-58         True
2      RestingECG  4.229233e-03         True
3  ExerciseAngina  2.907808e-50         True
4        ST_Slope  5.167638e-78         True


# Logistic Regression

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [52]:
# Separate features and target
X = df1.drop(columns=['HeartDisease'])  # Features
y = df1['HeartDisease']  # Target variable

In [53]:
# Identify categorical and numerical columns
categorical_features = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
numerical_features = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']

In [54]:
# Preprocessing pipeline
categorical_transformer = OneHotEncoder(drop='first', handle_unknown='ignore')  # One-hot encode categorical variables
numerical_transformer = StandardScaler()  # Standardize numerical variables

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [55]:
# Define the logistic regression model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

In [56]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [57]:
# Train the model
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'RestingBP',
                                                   'Cholesterol', 'FastingBS',
                                                   'MaxHR', 'Oldpeak']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['Sex', 'ChestPainType',
                                                   'RestingECG',
                                                   'ExerciseAngina',
                                                   'ST_Slope'])])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [58]:
# Make predictions on the test set
y_pred = model.predict(X_test)

In [59]:
model.score(X_train, y_train)

0.8692098092643051

In [60]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy * 100:.2f}%\n")

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))



Accuracy: 84.24%

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.87      0.82        77
           1       0.90      0.82      0.86       107

    accuracy                           0.84       184
   macro avg       0.84      0.85      0.84       184
weighted avg       0.85      0.84      0.84       184

Confusion Matrix:
[[67 10]
 [19 88]]


# DecisionTree

In [61]:
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
import matplotlib.pyplot as plt

In [62]:
# Create and train the DecisionTreeClassifier
clf = DecisionTreeClassifier( random_state=42)
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [63]:
# Evaluate the model
accuracy = clf.score(X_train, y_train)

In [64]:
accuracy

1.0

# RandomForest

In [65]:
from sklearn.ensemble import RandomForestClassifier

In [66]:
# Create and train the Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [67]:
# Make predictions on the test set
y_pred = rf_clf.predict(X_test)

In [68]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

In [69]:
accuracy

0.875

# AdaBoost Algorithm

In [70]:
from sklearn.ensemble import AdaBoostClassifier

In [71]:
# Initialize the AdaBoostClassifier with a DecisionTreeClassifier as the base estimator
base_estimator = DecisionTreeClassifier(max_depth=1)  # Weak learner
adaboost_clf = AdaBoostClassifier(estimator=base_estimator, n_estimators=50, learning_rate=1.0, random_state=42)

In [72]:
# Train the model
adaboost_clf.fit(X_train, y_train)

c:\Users\akhil\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   random_state=42)

In [73]:
# Predict on the test set
y_pred = adaboost_clf.predict(X_test)

In [74]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

In [75]:
accuracy

0.8532608695652174

# GradientBoostingClassifier 

In [76]:
from sklearn.ensemble import GradientBoostingClassifier

In [77]:
# Initialize the GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(
    n_estimators=100,  # Number of boosting stages
    learning_rate=0.1,  # Shrinkage factor for contributions
    max_depth=3,       # Maximum depth of each tree
    random_state=42
)

In [78]:
# Train the model
gb_clf.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

In [79]:
# Predict on the test set
y_pred = gb_clf.predict(X_test)

In [80]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

In [81]:
accuracy

0.875

# L1&L2 Classification

In [82]:
from sklearn.linear_model import LogisticRegression

In [83]:
# Logistic Regression with L1 regularization
logreg_l1 = LogisticRegression(penalty='l1', solver='saga', C=1.0, max_iter=1000, random_state=42)
logreg_l1.fit(X_train, y_train)


c:\Users\akhil\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=1000, penalty='l1', random_state=42, solver='saga')

In [84]:
# Logistic Regression with L2 regularization
logreg_l2 = LogisticRegression(penalty='l2', solver='lbfgs', C=1.0, max_iter=1000, random_state=42)
logreg_l2.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [85]:
# Evaluate the models
y_pred_l1 = logreg_l1.predict(X_test)
y_pred_l2 = logreg_l2.predict(X_test)

In [86]:
accuracy_score(y_test,y_pred_l1)

0.8260869565217391

In [87]:
accuracy_score(y_test,y_pred_l2)

0.8260869565217391